In [44]:
import pandas as pd
import numpy as np
import math
import datetime
from ipynb.fs.defs.functions import new_route
from ipynb.fs.defs.functions import get_row
from ipynb.fs.defs.functions import get_column
from ipynb.fs.defs.functions import best_index
from ipynb.fs.defs.functions import find_best_position
from ipynb.fs.defs.functions import cities_list
from ipynb.fs.defs.functions import getName

dft = pd.read_csv("../Dati/distanze/times_computed.csv", delimiter=";", encoding = "ISO-8859-1", index_col=0)
fur1 = pd.read_csv("../Dati/turni_f/2-opt1_shifts.csv", delimiter=";", encoding = "ISO-8859-1")
fur2 = pd.read_csv("../Dati/turni_f/2-opt2_shifts.csv", delimiter=";", encoding = "ISO-8859-1")

In [45]:
column = [getName(dft, x) for x in fur1["mercoledi"].dropna()]
pad = pd.Series([np.nan]*(len(fur1)-len(column)))
column = pd.concat([pd.Series(column), pad], ignore_index=True)
fur1["mercoledi"] = column

In [46]:
fur2

,martedi,mercoledi,giovedi
0,"Cremona, Sesto 39","Cremona, Sesto 39","Cremona, Sesto 39"
1,"Cremona, Felice Cavallotti, 2","Cremona, Palosca n. 2","Cremona, Ugolani Dati, 4"
2,"Pianengo, Nenni","Paderno Ponchielli, Municipio, 3","Pizzighettone, Giuseppe Garibaldi, 18"
3,"Offanengo, Clavelli Martini, 5","San Bassano, Mons. Frosi","Gadesco-Pieve Delmona, Giovanni Lonati 9"
4,"Manerbio, INSURREZIONE NAZIONALE","Fiesco, Giuseppe Parini, 10","Pescarolo ed Uniti, Giuseppe Mazzini, 73"
5,"Bonemerse, Roma, 25","Soncino, Dante Alighieri, 23","Cingia de' Botti, Giuseppina, 79"
6,"Sospiro, Tosi","Soresina, Giacomo Matteotti, 6","Martignana di Po, Libertà, 64"
7,"Gadesco-Pieve Delmona, Giovanni Lonati 9","Casalbuttano ed Uniti, Jacini, 23","Solarolo Rainerio, Giuseppina, 81"
8,"Vescovato, Roma, 44","San Giovanni in Croce, Feudatari, 25 A","San Giovanni in Croce, Feudatari, 25 A"
9,"Pescarolo ed Uniti, Giuseppe Mazzini, 73",NaN,NaN


In [47]:
#Given a list of cities it return False if "to_insert" is already present
#Return True if "to_insert" isn't present and it will give the index of the best position
def check_insert(schedule_day_list, to_insert, dframe):
    if to_insert in schedule_day_list:
        return False, -1
    else:
        #schedule_day_list = cities_list(new_route(schedule_day_list, dframe))
        to_return = best_index(schedule_day_list, new_route(schedule_day_list, dframe), to_insert, dframe)
        return True, to_return

#Given: city list wich to_insert come, the city "to_insert" to be moved, 
#the city list where "to_insert" will be insered, the index of the best position and a dataframe
#the function return the improvement
#If negative the exchange mustn't be done, if positive you gain some time
def improvement(f_l, to_insert, t_l, index, dframe):
    from_list = f_l[:]
    from_time = sum(new_route(from_list, dframe).values)
    from_list.remove(to_insert)
    from_time_removed = sum(new_route(from_list, dframe).values)
    
    to_list = t_l[:]
    to_time = sum(new_route(to_list, dframe).values)
    try:
        to_list.insert(index, to_insert)
    except:
        print(to_list)
    to_time_insert = sum(new_route(to_list, dframe).values)
    
    return (from_time-from_time_removed) + (to_time-to_time_insert)

In [48]:
def get_best_improvement(curr_day, from_vehicle, to_vehicle, best_improvement, best_day, best_vehicle, best_idx, to_insert, curr_f_list, fur1_copy, fur2_copy):
    local_improvement = ""
    if(to_vehicle==1):
        days = ["lunedi", "martedi", "mercoledi", "giovedi", "venerdi"]
    else:
        days = ["martedi", "mercoledi", "giovedi"]
        
    for inner_day in days:
        if(to_vehicle==1):
            curr_to_list = list(fur1_copy[inner_day].dropna())
        else:
            curr_to_list = list(fur2_copy[inner_day].dropna())
           
        #print(inner_day)
        if inner_day == curr_day and from_vehicle == to_vehicle:
            #print("continua")
            continue
        else:
            boo, index = check_insert(curr_to_list, to_insert, dft)
        if boo:
            local_improvement = improvement(curr_f_list, to_insert, curr_to_list, index, dft)
        else:
            continue

        #If there's an improvement we update "best" variables, in other case we skip to next day
        #print(local_improvement)
        if local_improvement<0:
            continue
        elif local_improvement>best_improvement:
            best_improvement = local_improvement
            best_day = inner_day
            best_vehicle = to_vehicle
            best_idx = index
        else:
            continue
    return best_improvement, best_day, best_vehicle, best_idx

In [49]:
#For every day of every veichle we get every Location
#We compare the improvement removing this location from the day/veichle routing list and inserting it 
#in the other day/veichle routing lists.
#The location will be moved only if the improvement is positive 
#In case of different positive improvement we move in the higher
pudding = 25
week = ["lunedi", "martedi", "mercoledi", "giovedi", "venerdi"]
short_week = ["martedi", "mercoledi", "giovedi"]
fur1_copy = pd.DataFrame({})
fur1_copy['col'] = pd.Series([0]*pudding)
fur1_copy = fur1_copy.join(fur1)

fur2_copy = pd.DataFrame({})
fur2_copy['col'] = pd.Series([0]*pudding)
fur2_copy = fur2_copy.join(fur2)

for day in week:
    #day="lunedi"
    print(day, end="###\n")
    curr_from_list = list(fur1_copy[day].dropna())
    to_insert_list = list(fur1_copy[day].dropna())
    #print(to_insert_list)
    #to_insert_list.remove("Ostiano, Mazzini, 1")
    #curr_from_list.remove("Ostiano, Mazzini, 1")
    cycles = len(to_insert_list)
    #For every location in this schedule
    for i in range(1, cycles-1):
        #print(to_insert_list)
        to_insert = to_insert_list.pop(0)
        #print(to_insert, end=", ")

        best_improvement = -1000
        best_day = ""
        best_vehicle = ""
        best_idx = ""
        #Here we must compare every possible insert in other days
        #Cycle for first veichle
        best_improvement, best_day, best_vehicle, best_idx = get_best_improvement(day, 1, 1, best_improvement, best_day, best_vehicle, best_idx, to_insert, curr_from_list, fur1_copy, fur2_copy)
        best_improvement, best_day, best_vehicle, best_idx = get_best_improvement(day, 1, 2, best_improvement, best_day, best_vehicle, best_idx, to_insert, curr_from_list, fur1_copy, fur2_copy)

        #If best_day isn't modified where isn't an improvement for every day/veichle
        if(best_day==""):
            continue
        else:
            #Remove from origin list and insert into target list
            curr_from_list.remove(to_insert)
            
            if(best_vehicle==1):
                #print(f"{to_insert} miglioramento veicolo 1 a {best_day}")
                temp_list = list(fur1_copy[best_day].dropna())
                #temp_list.append("Ostiano, Mazzini, 1")
                padding = pd.Series([np.nan]*(pudding-len(temp_list)))
                temp_list = pd.concat([pd.Series(temp_list), padding], ignore_index=True)
                #fur1_copy[best_day] = fur1_copy[best_day].replace(list(fur1_copy[best_day]), temp_list)
                fur1_copy[best_day] = pd.Series(temp_list).values

            elif(best_vehicle==2):
                #print(f"{to_insert} miglioramento veicolo 2 a {best_day}")
                temp_list = list(fur2_copy[best_day].dropna())
                temp_list.insert(best_idx, to_insert)
                #temp_list.append("Ostiano, Mazzini, 1")
                padding = pd.Series([np.nan]*(pudding-len(temp_list)))
                temp_list = pd.concat([pd.Series(temp_list), padding], ignore_index=True)
                #fur2_copy[best_day] =  fur2_copy[best_day].replace(list(fur2_copy[best_day]), temp_list)
                fur2_copy[best_day] = pd.Series(temp_list).values
    padding = pd.Series([np.nan]*(pudding-len(curr_from_list)))
    curr_from_list = pd.concat([pd.Series(curr_from_list), padding], ignore_index=True)
    fur1_copy[day] = pd.Series(curr_from_list).values
fur1_copy   

lunedi###
martedi###
mercoledi###
giovedi###
venerdi###


,col,lunedi,martedi,mercoledi,giovedi,venerdi
0,0,"Cremona, Sesto 39","Cremona, Sesto 39","Cremona, Sesto 39","Cremona, Sesto 39","Cremona, Sesto 39"
1,0,"Sesto ed Uniti, Roma, 4","Scandolara Ravara, Italia","Castelverde, Ubaldo Ferrari, 15","Cremona, Sesto 39","Crema, Civerchi, 9"
2,0,"Grumello Cremonese ed Uniti, Roma, 2","San Giovanni in Croce, Feudatari, 25 A","Castelleone, Roma, 67","Persico Dosimo, Albert ,7","Casaletto Vaprio, Maggiore, 7"
3,0,"Montodine, XXV Aprile, 10","Ostiano, Mazzini, 1","Madignano, Portici, 4","San Giovanni in Croce, Feudatari, 25 A","Trescore Cremasco, Vittorio Veneto, 1"
4,0,"Ripalta Cremasca, Roma, 5","Ostiano, Mazzini, 1","Bagnolo Cremasco, Roma, 3","Piadena, Platina, 40","Pandino, Castello, 15"
5,0,"Soncino, Dante Alighieri, 23",NaN,"Palazzo Pignano, Marco Polo, 31","Ostiano, Mazzini, 1","Rivolta d'Adda, Stefano Renzi, 5"
6,0,"Soresina, Giacomo Matteotti, 6",NaN,"Pandino, Castello, 15","Ostiano, Mazzini, 1","Spino d'Adda, Circonvallazione, 13"
7,0,"Ostiano, Mazzini, 1",NaN,"Rivolta d'Adda, Stefano Renzi, 5",NaN,"Dovera, Oldrini 2"
8,0,NaN,NaN,"Spino d'Adda, Circonvallazione, 13",NaN,"Monte Cremasco, Roma, 12"
9,0,NaN,NaN,"Romanengo, Guaiarini, 14",NaN,"Vaiano Cremasco, Gloriosi Caduti, 3"


In [50]:
#For every day of every veichle we get every Location
#We compare the improvement removing this location from the day/veichle routing list and inserting it 
#in the other day/veichle routing lists.
#The location will be moved only if the improvement is positive 
#In case of different positive improvement we move in the higher
pudding = 25

for day in short_week:
    #day="lunedi"
    print(day, end="###\n")
    curr_from_list = list(fur2_copy[day].dropna())
    to_insert_list = list(fur2_copy[day].dropna())
    #print(to_insert_list)
    #to_insert_list.remove("Ostiano, Mazzini, 1")
    #curr_from_list.remove("Ostiano, Mazzini, 1")
    cycles = len(to_insert_list)
    #For every location in this schedule
    for i in range(1, cycles-1):
        #print(to_insert_list)
        to_insert = to_insert_list.pop(0)
        #print(to_insert, end=", ")
        #to_insert = to_insert_list.pop(0)

        best_improvement = -1000
        best_day = ""
        best_veichle = ""
        best_idx = ""
        #Here we must compare every possible insert in other days
        #Cycle for first veichle
        best_improvement, best_day, best_veichle, best_idx = get_best_improvement(day, 2, 1, best_improvement, best_day, best_veichle, best_idx, to_insert, curr_from_list, fur1_copy, fur2_copy)
        best_improvement, best_day, best_veichle, best_idx = get_best_improvement(day, 2, 2, best_improvement, best_day, best_veichle, best_idx, to_insert, curr_from_list, fur1_copy, fur2_copy)

        #If best_day isn't modified where isn't an improvement for every day/veichle
        if(best_day==""):
            continue
        else:
            #Remove from origin list and insert into target list
            curr_from_list.remove(to_insert)
            
            if(best_veichle==1):
                #print(f"{to_insert} miglioramento veicolo 1 a {best_day}")
                temp_list = list(fur1_copy[best_day].dropna())
                #temp_list.append("Ostiano, Mazzini, 1")
                padding = pd.Series([np.nan]*(pudding-len(temp_list)))
                temp_list = pd.concat([pd.Series(temp_list), padding], ignore_index=True)
                #fur1_copy[best_day] = fur1_copy[best_day].replace(list(fur1_copy[best_day]), temp_list)
                fur1_copy[best_day] = pd.Series(temp_list).values

            elif(best_veichle==2):
                #print(f"{to_insert} miglioramento veicolo 2 a {best_day}")
                temp_list = list(fur2_copy[best_day].dropna())
                temp_list.insert(best_idx, to_insert)
                #temp_list.append("Ostiano, Mazzini, 1")
                padding = pd.Series([np.nan]*(pudding-len(temp_list)))
                temp_list = pd.concat([pd.Series(temp_list), padding], ignore_index=True)
                #fur2_copy[best_day] =  fur2_copy[best_day].replace(list(fur2_copy[best_day]), temp_list)
                fur2_copy[best_day] = pd.Series(temp_list).values
    padding = pd.Series([np.nan]*(pudding-len(curr_from_list)))
    curr_from_list = pd.concat([pd.Series(curr_from_list), padding], ignore_index=True)
    fur2_copy[day] = pd.Series(curr_from_list).values
fur2_copy  

martedi###
mercoledi###
giovedi###


,col,martedi,mercoledi,giovedi
0,0,"Cremona, Sesto 39","Cremona, Sesto 39","Cremona, Sesto 39"
1,0,"Pianengo, Nenni","Paderno Ponchielli, Municipio, 3","Cremona, Felice Cavallotti, 2"
2,0,"Sergnano, IV Novembre, 10","Annicco, Giuseppe Garibaldi, 6","Gadesco-Pieve Delmona, Giovanni Lonati 9"
3,0,"Camisano, Trieste, 57","San Bassano, Mons. Frosi","Vescovato, Roma, 44"
4,0,"Casale Cremasco-Vidolasco, Roma, 56","Castelleone, Roma, 67","Cingia de' Botti, Giuseppina, 79"
5,0,"Ricengo, Guglielmo Marconi","Fiesco, Giuseppe Parini, 10","Martignana di Po, Libertà, 64"
6,0,"Gadesco-Pieve Delmona, Giovanni Lonati 9","Soncino, Dante Alighieri, 23","Gussola, Comaschi, 1"
7,0,"Vescovato, Roma, 44","Casalbuttano ed Uniti, Jacini, 23","Casalmaggiore, Guglielmo Marconi, 8"
8,0,"Rivarolo del Re ed Uniti, Giovanni XXIII, 5","San Giovanni in Croce, Feudatari, 25 A","Solarolo Rainerio, Giuseppina, 81"
9,0,"Viadana, A. Manzoni, 2",NaN,"San Giovanni in Croce, Feudatari, 25 A"


In [10]:
fur1_copy.to_csv("../Dati/turni_f/multi_veich_LS1.csv", sep=";", index=False)
fur2_copy.to_csv("../Dati/turni_f/multi_veich_LS2.csv", sep=";", index=False)

In [ ]:
fur2_copy = pd.DataFrame({})
fur2_copy['col'] = pd.Series([np.nan]*50)
fur2_copy = fur2_copy.join(fur2)
fur2_copy[["martedi", "mercoledi", "giovedi"]]

In [ ]:
fur1_copy = pd.DataFrame({})
fur1_copy['col'] = pd.Series([0]*50)
fur1_copy = fur1_copy.join(fur1)
temp_list = pd.Series(["ciao"])
padding=pd.Series([np.nan]*(50-len(temp_list)))
temp_list = pd.concat([temp_list, padding], ignore_index=True)
fur1_copy[best_day] = fur1_copy[best_day].replace(list(fur1_copy[best_day]), temp_list)